In [ ]:
import pandas as pd
import ast
import torch
from datasets import load_dataset
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import Trainer
import random
from transformers import pipeline
from rouge import Rouge
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
df = pd.read_csv("/home/recipes_small.csv") #30k version
#df = df.sample(5000, random_state=42) #change based on the sample size
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
# to make sure ingredients/directions are lists
def ensure_list(x):
    try:
        out = ast.literal_eval(x)
        # strip extra quotes & spaces
        if isinstance(out, list):
            return [str(i).strip().strip('"').strip("'") for i in out]
        else:
            return [str(out).strip().strip('"').strip("'")]
    except:
        return [str(x).strip().strip('"').strip("'")]


df["ingredients"] = df["ingredients"].apply(ensure_list)
df["directions"] = df["directions"].apply(ensure_list)

# Drop empty recipes
df = df[df["ingredients"].apply(len) > 0]
df = df[df["directions"].apply(len) > 0]
df = df[df["title"].str.strip().str.len() > 0]

In [ ]:
df.head(5)

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","[In a slow cooker, combine all ingredients. Co...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...","[Boil and debone chicken., Put bite size piece...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",[Combine first four ingredients and press in 1...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
def build_recipe(row):
    ingredients = "\n".join(f"- {i}" for i in row["ingredients"])
    steps = "\n".join(f"{idx+1}. {s}" for idx, s in enumerate(row["directions"]))

    text = (
        "<|RECIPE|>\n"
        f"Title: {row['title'].strip()}\n"
        "Ingredients:\n"
        f"{ingredients}\n\n"
        "Directions:\n"
        f"{steps}\n\n"
        "<|END|>\n"
    )
    return text

df["text"] = df.apply(build_recipe, axis=1)

In [ ]:
df.head(5)

,Unnamed: 0,title,ingredients,directions,link,source,NER,text
0,0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evapor...","[In a heavy 2-quart saucepan, mix brown sugar,...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",<|RECIPE|>\nTitle: No-Bake Nut Cookies\nIngred...
1,1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chi...","[Place chipped beef on bottom of baking dish.,...",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...",<|RECIPE|>\nTitle: Jewell Ball'S Chicken\nIngr...
2,2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...","[In a slow cooker, combine all ingredients. Co...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...",<|RECIPE|>\nTitle: Creamy Corn\nIngredients:\n...
3,3,Chicken Funny,"[1 large whole chicken, 2 (10 1/2 oz.) cans ch...","[Boil and debone chicken., Put bite size piece...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...",<|RECIPE|>\nTitle: Chicken Funny\nIngredients:...
4,4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker cru...",[Combine first four ingredients and press in 1...,www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu...",<|RECIPE|>\nTitle: Reeses Cups(Candy)\nIngredi...


In [ ]:
df.shape

(30000, 8)

In [ ]:
#conver to a HuggingFace Dataset
dataset = Dataset.from_pandas(df[["text"]])

In [ ]:
#train/val split
dataset = dataset.train_test_split(test_size=0.1, seed=42)

In [ ]:
#for later in evaluation part:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
# GPT2 and tokens

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

model = GPT2LMHeadModel.from_pretrained("gpt2")

special_tokens = {
    "additional_special_tokens": ["<|RECIPE|>", "<|END|>"]
}
tokenizer.add_special_tokens(special_tokens)

model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [ ]:
#tokenizing the data
tokenizer.pad_token = tokenizer.eos_token
def tokenize(batch):
  return tokenizer(
      batch["text"],
      #padding="max_length",
      truncation=True,
      max_length=512,
      padding=False
  )

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
#data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, #bc gpt2 isn't masked-lm
)

In [ ]:
#training arg
training_args = TrainingArguments(
    output_dir="./gpt2-recipe",
    overwrite_output_dir=True,

    eval_strategy="steps",     # Early stopping
    eval_steps=500,
    save_steps=500,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,


    num_train_epochs=100, #training
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    warmup_steps=100,

    logging_steps=100,
    save_strategy="steps",
    save_total_limit=2,
    fp16=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
#train by HuggingFace Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]

)
trainer.train()

Step,Training Loss,Validation Loss
500,1.646500,1.554171
1000,1.553200,1.482362
1500,1.513000,1.440710
2000,1.457900,1.411681
2500,1.452800,1.389845
3000,1.418100,1.372196
3500,1.383900,1.356023
4000,1.373600,1.345577
4500,1.343600,1.333975
5000,1.350100,1.325873


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=23000, training_loss=1.2641207179193912, metrics={'train_runtime': 6548.4042, 'train_samples_per_second': 412.314, 'train_steps_per_second': 12.889, 'total_flos': 8.9580168105984e+16, 'train_loss': 1.2641207179193912, 'epoch': 27.25125925925926})

In [ ]:
#save model
trainer.save_model("/home/gpt2-recipe_final3")
tokenizer.save_pretrained("/home/gpt2-recipe_final3")

('/home/gpt2-recipe_final3/tokenizer_config.json',
 '/home/gpt2-recipe_final3/special_tokens_map.json',
 '/home/gpt2-recipe_final3/vocab.json',
 '/home/gpt2-recipe_final3/merges.txt',
 '/home/gpt2-recipe_final3/added_tokens.json')

In [ ]:
model_path = "/home/gpt2-recipe_final3"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

In [ ]:
test_titles = df_test["title"].tolist()
random.shuffle(test_titles)
test_titles = test_titles[:100]

In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0
)
def generate_recipe(title, ingredients, mode):
    prompt = f"<|RECIPE|>\nTitle: {title}\nIngredients:\n"
    prompt += "\n".join(f"- {i}" for i in ingredients)
    prompt += "\n\nDirections:\n"

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # stop at <|END|>
    eos_id = tokenizer.convert_tokens_to_ids("<|END|>")

    if mode == "greedy":
        params = dict(
            do_sample=False,
            num_beams=1,
        )

    elif mode == "beam":
        params = dict(
            do_sample=False,
            num_beams=4,            
        )

    elif mode == "topk":
        params = dict(
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.8,
            num_beams=1,
        )

    else:
        raise ValueError("Unknown mode")

    output = model.generate(
        input_ids,
        eos_token_id=eos_id,       ###
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=300,
        **params
    )

    # decode
    text = tokenizer.decode(output[0], skip_special_tokens=False)

    # clean
    if "<|END|>" in text:
        text = text.split("<|END|>")[0]

    return text.strip()


Device set to use cuda:0


In [ ]:
rouge = Rouge()
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def evaluate_strategy(mode):
    scores_rouge1 = []
    scores_rougeL = []
    scores_cosine = []
    results = []    

    for title in test_titles:
        ref_row = df_test[df_test["title"] == title].iloc[0]
        ref_ingredients = ref_row["ingredients"]
        ref_text = ref_row["text"]

        gen_text = generate_recipe(title, ref_ingredients, mode=mode)

        score = rouge.get_scores(gen_text, ref_text)[0]
        r1 = score["rouge-1"]["f"]
        rl = score["rouge-l"]["f"]

        e1 = embedder.encode(gen_text, convert_to_tensor=True)
        e2 = embedder.encode(ref_text, convert_to_tensor=True)
        cosine = float(util.pytorch_cos_sim(e1, e2)[0][0])

        scores_rouge1.append(r1)
        scores_rougeL.append(rl)
        scores_cosine.append(cosine)

        results.append({
            "title": title,
            "generated": gen_text,
            "reference": ref_text,
            "cosine": cosine
        })

    # sort best & worst by cosine similarity
    results_sorted = sorted(results, key=lambda x: x["cosine"], reverse=True)
    best3 = results_sorted[:3]
    worst3 = results_sorted[-3:]

    return {
        "rouge1": np.mean(scores_rouge1),
        "rougeL": np.mean(scores_rougeL),
        "cosine": np.mean(scores_cosine),
        "best3": best3,
        "worst3": worst3
    }

In [ ]:
results_greedy = evaluate_strategy("greedy")
results_beam   = evaluate_strategy("beam")
results_topk   = evaluate_strategy("topk")

##rouge1
print("Rouge-1:")
print(results_greedy["rouge1"], results_greedy["cosine"])
print(results_beam["rouge1"],   results_beam["cosine"])
print(results_topk["rouge1"],   results_topk["cosine"])

##rougeL
print("Rouge-L:")
print(results_greedy["rougeL"], results_greedy["cosine"])
print(results_beam["rougeL"],   results_beam["cosine"])
print(results_topk["rougeL"],   results_topk["cosine"])

print("Best greedy examples:", results_greedy["best3"])
print("Worst beam examples:", results_beam["worst3"])


Rouge-1:
0.7020721181538359 0.9617004817724228
0.6958951651661998 0.9560064244270324
0.6699427618355638 0.9538248199224472
Rouge-L:
0.6991618514040573 0.9617004817724228
0.6914792840377977 0.9560064244270324
0.6664040720498128 0.9538248199224472
Best greedy examples: [{'title': 'Hearty Sausage Stew', 'generated': '<|RECIPE|> \nTitle: Hearty Sausage Stew\nIngredients:\n- 1 1/2 to 2 lb. Polish sausage\n- 2 (16 oz.) cans Great Northern beans (undrained)\n- 3 c. coarsely chopped cabbage\n- 2 c. water *\n- 1 1/2 c. sliced carrots (1-inch pieces)\n- 2 chicken bouillon cubes\n- 3/4 c. sliced celery (1/2-inch pieces)\n- 1 Tbsp. minced onion (I use a whole onion)\n- 1/2 tsp. basil\n- 1/4 tsp. dried thyme leaves\n- 1/4 tsp. dried summer savory leaves (optional)\n- 1/8 tsp. minced garlic or garlic salt\n\nDirections:\n1. *I use 1/2 cup water.', 'reference': '<|RECIPE|>\nTitle: Hearty Sausage Stew\nIngredients:\n- 1 1/2 to 2 lb. Polish sausage\n- 2 (16 oz.) cans Great Northern beans (undrained)\n-

In [ ]:
title = "Banana Bread"
ingredients = ["1 cup peanut butter", "1/2 cup honey", "2 cups oats", "1/2 cup chocolate chips"]

recipe = generate_recipe(title, ingredients, mode="greedy")
print(recipe)

recipe = generate_recipe(title, ingredients, mode="beam")
print(recipe)

recipe = generate_recipe(title, ingredients, mode="topk")
print(recipe)

<|RECIPE|> 
Title: Banana Bread
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Mix all ingredients together.
2. Bake at 350 degrees for 45 minutes.
<|RECIPE|> 
Title: Banana Bread
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Mix peanut butter, honey, oats and chocolate chips.
2. Pour into greased loaf pan.
3. Bake at 350 degrees for 1 hour.
<|RECIPE|> 
Title: Banana Bread
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Mix together peanut butter, honey and oats.
2. Pour into 2 loaf pans.
3. Bake at 350 degrees for 45 minutes.


In [ ]:
title = "Chocolate chip coocie"
ingredients = ["1 cup peanut butter", "1/2 cup honey", "2 cups oats", "1/2 cup chocolate chips"]

recipe = generate_recipe(title, ingredients, mode="greedy")
print(recipe)

recipe = generate_recipe(title, ingredients, mode="beam")
print(recipe)

recipe = generate_recipe(title, ingredients, mode="topk")
print(recipe)

<|RECIPE|> 
Title: Chocolate chip coocie
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Mix together and press into a 9x13 pan.
2. Bake at 350 degrees for 15 minutes.
<|RECIPE|> 
Title: Chocolate chip coocie
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Mix peanut butter, honey, oats and chocolate chips together.
2. Pour into greased 9x13 pan.
3. Bake at 350 degrees for 30 minutes.
<|RECIPE|> 
Title: Chocolate chip coocie
Ingredients:
- 1 cup peanut butter
- 1/2 cup honey
- 2 cups oats
- 1/2 cup chocolate chips

Directions:
1. Combine all ingredients in saucepan. Heat until mixture begins to boil. Remove from heat. Stir in chocolate chips. Pour into buttered pan. Cool.


In [ ]:
#!zip -r gpt2-recipe-final.zip /home/gpt2-recipe_final3
#from google.colab import files
#files.download('gpt2-recipe-final.zip')

  adding: home/gpt2-recipe_final3/ (stored 0%)
  adding: home/gpt2-recipe_final3/config.json (deflated 51%)
  adding: home/gpt2-recipe_final3/special_tokens_map.json (deflated 77%)
  adding: home/gpt2-recipe_final3/added_tokens.json (deflated 22%)
  adding: home/gpt2-recipe_final3/generation_config.json (deflated 24%)
  adding: home/gpt2-recipe_final3/model.safetensors (deflated 7%)
  adding: home/gpt2-recipe_final3/vocab.json (deflated 68%)
  adding: home/gpt2-recipe_final3/tokenizer_config.json (deflated 69%)
  adding: home/gpt2-recipe_final3/training_args.bin (deflated 53%)
  adding: home/gpt2-recipe_final3/merges.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>